### Author: Bongsang Kim
- homepage: https://bongsang.github.io
- Linkedin: https://www.linkedin.com/in/bongsang

# Face landmarks detection by CNN

### Step 1. Data Analysis

The first step is to load in the images of faces and their keypoints and visualize them.
This set of image data has been extracted from the [YouTube Faces Dataset](https://www.cs.tau.ac.il/~wolf/ytfaces/), which includes videos of people in YouTube videos. These videos have been fed through some processing steps and turned into sets of image frames containing one face and the associated keypoints.

<img src="https://www.cs.tau.ac.il/~wolf/ytfaces/logo.jpg" width=50%>

#### Training and Testing Data

This facial landmarks dataset consists of 5,770 images. All of these images are separated into either a training or a test set of data.

Total | Training | Testing
----- | -------- | -------
5,770 | 3,462    | 2,308
100%  | 60%      | 40%

The information about the images and keypoints in this dataset are summarized in CSV files, which we can read in using `pandas`. Let's read the training CSV and get the annotations in an (N, 2) array where N is the number of keypoints and 2 is the dimension of the keypoint coordinates (x, y).

---

In [ ]:
import pandas as pd
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt


# import glob

# import skimage.io as io
# from torch.utils.data import DataLoader
# from torchvision import transforms, utils

# # Bongsang's personal pacakges
# from landmarks_dataset import LandmarksDataset
# from landmarks_transform import Rescale, RandomCrop, Normalize, ToTensor

%matplotlib inline

In [ ]:
landmarks_frame = pd.read_csv('data/training_landmarks.csv')
print(f'The number of training dataset is {len(landmarks_frame)}. {3462/5770*100}% of 5,770')
landmarks_frame.head()

#### Plotting some images from data frame

In [ ]:
def get_image_name(frame, idx):
    return frame.iloc[idx, 0]

def get_landmarks(frame, idx):
    landmark_points = frame.iloc[idx, 1:].to_numpy()
    landmark_points = landmark_points.astype('float').reshape(-1, 2)
    return landmark_points

def show_landmarks(image, points, name):
    plt.title(name)
    plt.imshow(image)
    plt.scatter(points[:, 0], points[:, 1], s=20, marker='.', c='m')
    plt.pause(0.1)

In [ ]:
for _ in range(2):
    idx = np.random.randint(0, len(landmarks_frame))
    image_name = get_image_name(landmarks_frame, idx)
    image_path = os.path.join('data/training/', image_name)
    image = cv2.imread(image_path)
    image_data = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    landmarks = get_landmarks(landmarks_frame, idx)
    show_landmarks(image_data, landmarks, image_name)

## Dataset class and Transformations

To prepare our data for training, we'll be using PyTorch's Dataset class. Much of this this code is a modified version of what can be found in the [PyTorch data loading tutorial](http://pytorch.org/tutorials/beginner/data_loading_tutorial.html).

<img src="https://pytorch.org/tutorials/_images/landmarked_face2.png" width=25%>

### PyTorch Dataset class

``torch.utils.data.Dataset`` is an abstract class representing a
dataset. This class will allow us to load batches of image/keypoint data, and uniformly apply transformations to our data, such as rescaling and normalizing images for training a neural network.


Your custom dataset should inherit ``Dataset`` and override the following
methods:

-  ``__len__`` so that ``len(dataset)`` returns the size of the dataset.
-  ``__getitem__`` to support the indexing such that ``dataset[i]`` can
   be used to get the i-th sample of image/keypoint data.

Let's create a dataset class for our face keypoints dataset. We will
read the CSV file in ``__init__`` but leave the reading of images to
``__getitem__``. This is memory efficient because all the images are not
stored in the memory at once but read as required.

A sample of our dataset will be a dictionary
``{'image': image, 'keypoints': key_pts}``. Our dataset will take an
optional argument ``transform`` so that any required processing can be
applied on the sample. We will see the usefulness of ``transform`` in the
next section.


In [ ]:
import torch
from torch.utils.data import Dataset

class LandmarksDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, csv_file, root_dir, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied on a sample.
        """
        self.landmarks_frame = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.landmarks_frame)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        image_name = self.landmarks_frame.iloc[idx, 0]
        image_path = os.path.join(self.root_dir, image_name)
        image = cv2.imread(image_path)
        image_data = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        landmarks = self.landmarks_frame.iloc[idx, 1:]
        landmarks = np.array([landmarks])
        landmarks = landmarks.astype('float').reshape(-1, 2)
        sample = {'image': image_data, 'landmarks': landmarks, 'name': image_name}
        print(f'LandmarksDataset shape, file={image_name}, image={image_data.shape}, landmarks={landmarks.shape}')


        if self.transform:
            sample = self.transform(sample)

        return sample

#### Plotting some images from PyTorch dataset

In [ ]:
landmarks_dataset = LandmarksDataset(csv_file='data/training_landmarks.csv',
                                    root_dir='data/training',
                                    transform=None)
for _ in range(2):
    idx = np.random.randint(0, len(landmarks_dataset))
    sample = landmarks_dataset[idx]
    show_landmarks(sample['image'], sample['landmarks'], sample['name'])

## PyTorch Transforms

Now, the images above are not of the same size, and neural networks often expect images that are standardized; a fixed size, with a normalized range for color ranges and coordinates, and (for PyTorch) converted from numpy lists and arrays to Tensors.

Therefore, we will need to write some pre-processing code.
Let's create four transforms:

-  ``Normalize``: to convert a color image to grayscale values with a range of [0,1] and normalize the keypoints to be in a range of about [-1, 1]
-  ``Rescale``: to rescale an image to a desired size.
-  ``RandomCrop``: to crop an image randomly.
-  ``ToTensor``: to convert numpy images to torch images.


We will write them as callable classes instead of simple functions so
that parameters of the transform need not be passed everytime it's
called. For this, we just need to implement ``__call__`` method and 
(if we require parameters to be passed in), the ``__init__`` method. 
We can then use a transform like this:

    tx = Transform(params)
    transformed_sample = tx(sample)

Observe below how these transforms are generally applied to both the image and its keypoints.



In [ ]:
class Normalize(object):
    """Convert a color image to grayscale and normalize the color range to [0,1]."""

    def __call__(self, sample):
        image, landmarks, name = sample['image'], sample['landmarks'], sample['name']


        image_copy = np.copy(image)
        landmarks_copy = np.copy(landmarks)

        # convert image to grayscale
        image_copy = cv2.cvtColor(image_copy, cv2.COLOR_RGB2GRAY)

        # scale color range from [0, 255] to [0, 1]
        image_copy = image_copy / 255.0

        # scale landmarks to be centered around 0 with a range of [-1, 1]
        # mean = 100, sqrt = 50, so, pts should be (pts - 100)/50
        landmarks_copy = (landmarks_copy - 100) / 50.0

        print(f'Transformed LandmarksDataset shape, file={name}, rescaled shape = {image_copy.shape}')
        
        return {'image':image_copy, 'landmarks':landmarks_copy, 'name': name}



class Rescale(object):
    """Rescale the image in a sample to a given size.

    Args:
        output_size (tuple or int): Desired output size. If tuple, output is
            matched to output_size. If int, smaller of image edges is matched
            to output_size keeping aspect ratio the same.
    """

    def __init__(self, output_size):
        assert isinstance(output_size, (int, tuple))
        self.output_size = output_size

    def __call__(self, sample):
        image, landmarks, name = sample['image'], sample['landmarks'], sample['name']

        h, w = image.shape[:2]
        if isinstance(self.output_size, int):
            if h > w:
                new_h, new_w = self.output_size * h / w, self.output_size
            else:
                new_h, new_w = self.output_size, self.output_size * w / h
        else:
            new_h, new_w = self.output_size

        new_h, new_w = int(new_h), int(new_w)

        image_copy = cv2.resize(image, (new_w, new_h))
        landmarks_copy = landmarks * [new_w / w, new_h / h]
        
        print(f'Rescaled shape, file={name}, rescaled shape = {image_copy.shape}')

        return {'image':image_copy, 'landmarks':landmarks_copy, 'name': name}


class RandomCrop(object):
    """Crop randomly the image in a sample.

    Args:
        output_size (tuple or int): Desired output size. If int, square crop
            is made.
    """

    def __init__(self, output_size):
        assert isinstance(output_size, (int, tuple))
        if isinstance(output_size, int):
            self.output_size = (output_size, output_size)
        else:
            assert len(output_size) == 2
            self.output_size = output_size

    def __call__(self, sample):
        image, landmarks, name = sample['image'], sample['landmarks'], sample['name']

        h, w = image.shape[:2]
        new_h, new_w = self.output_size

        top = np.random.randint(0, h - new_h)
        left = np.random.randint(0, w - new_w)

        image_copy = image[top: top + new_h, left: left + new_w]
        landmarks_copy = landmarks - [left, top]

        print(f'Croped shape, file={name}, rescaled shape = {image_copy.shape}')

        return {'image':image_copy, 'landmarks':landmarks_copy, 'name': name}



class ToTensor(object):
    """Convert ndarrays in sample to Tensors."""

    def __call__(self, sample):
        image, landmarks, name = sample['image'], sample['landmarks'], sample['name']

        # swap color axis because
        # numpy image: H x W x C
        # torch image: C X H X W
        image_copy = image.transpose((2, 0, 1))
        print(f'Transformed LandmarksDataset shape, file={name}, rescaled shape = {image_copy.shape}')

        return {'image': torch.from_numpy(image_copy), 'landmarks': torch.from_numpy(landmarks), 'name': name}

### Test out the transforms: Rescale any images to 250 size (height)

In [ ]:
from torchvision import transforms
rescale = Rescale(250)

idx = np.random.randint(0, len(landmarks_dataset))
sample = landmarks_dataset[idx]
transformed_sample = rescale(sample)
show_landmarks(transformed_sample['image'], transformed_sample['landmarks'], transformed_sample['name'])

### Test out the transforms: Crop any images to 50 size

In [ ]:
from torchvision import transforms
crop = RandomCrop(80)

idx = np.random.randint(0, len(landmarks_dataset))
sample = landmarks_dataset[idx]
transformed_sample = crop(sample)
show_landmarks(transformed_sample['image'], transformed_sample['landmarks'], transformed_sample['name'])

### Test out the transforms: Rescale to 250 size and then crop 224 size

In [ ]:
from torchvision import transforms
composed = transforms.Compose([Rescale(250), RandomCrop(224)])

idx = np.random.randint(0, len(landmarks_dataset))
sample = landmarks_dataset[idx]
transformed_sample = composed(sample)
show_landmarks(transformed_sample['image'], transformed_sample['landmarks'], transformed_sample['name'])

## Create the transformed dataset

Apply the transforms in order to get grayscale images of the same shape. Verify that your transform works by printing out the shape of the resulting data (printing out a few examples should show you a consistent tensor size).

In [ ]:
# define the data tranform
# order matters! i.e. rescaling should come before a smaller crop
data_transform = transforms.Compose([Rescale(250),
                                     RandomCrop(224),
                                     Normalize(),
                                     ToTensor()])

# create the transformed dataset
transformed_dataset = FacialKeypointsDataset(csv_file='data/training_frames_keypoints.csv',
                                             root_dir='data/training/',
                                             transform=data_transform)


In [ ]:
# print some stats about the transformed data
print('Number of images: ', len(transformed_dataset))

# make sure the sample tensors are the expected size
for i in range(5):
    sample = transformed_dataset[i]
    print(i, sample['image'].size(), sample['keypoints'].size())


## Data Iteration and Batching

Right now, we are iterating over this data using a ``for`` loop, but we are missing out on a lot of PyTorch's dataset capabilities, specifically the abilities to:

-  Batch the data
-  Shuffle the data
-  Load the data in parallel using ``multiprocessing`` workers.

``torch.utils.data.DataLoader`` is an iterator which provides all these
features, and we'll see this in use in the *next* notebook, Notebook 2, when we load data in batches to train a neural network!

---



## Ready to Train!

Now that you've seen how to load and transform our data, you're ready to build a neural network to train on this data.

In the next notebook, you'll be tasked with creating a CNN for facial keypoint detection.